In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/"My Drive"/"Master Thesis"/"Model"

/content/drive/My Drive/Master Thesis/Model


Importing dependencies numpy and keras

In [3]:
import os
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, LSTM, SimpleRNN, Embedding, LeakyReLU, concatenate, BatchNormalization, TimeDistributed
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


生成時間序列資料 load traffic data

In [4]:
data_dir = "../Data"

x, y = None, None   # 初始化

for env_folder in os.listdir(data_dir):
    print(env_folder)
    # 檢查.npy檔
    if os.path.isfile(os.path.join(data_dir, env_folder, 'X.npy')) and os.path.isfile(os.path.join(data_dir, env_folder, 'Y.npy')):
        env_x = np.load(os.path.join(data_dir, env_folder, 'X.npy'), allow_pickle=True)
        env_y = np.load(os.path.join(data_dir, env_folder, 'Y.npy'), allow_pickle=True)    # 從 .npy 檔載入已經算好的數據
        print("env_x.shape:", env_x.shape)
        print("env_y.shape:", env_y.shape)
        if x is None or y is None:
            x = env_x.copy()
            y = env_y.copy()
        else:
            x = np.vstack([x, env_x])
            y = np.vstack([y, env_y])

print("x.shape:", x.shape)
print("y.shape:", y.shape)

01-section
env_x.shape: (5145, 5, 61)
env_y.shape: (5145, 1, 2)
02-section
env_x.shape: (3362, 5, 61)
env_y.shape: (3362, 1, 2)
x.shape: (8507, 5, 61)
y.shape: (8507, 1, 2)


In [0]:
print(x[0,0,0:2])
print(x[0,0,2:13])
print(x[0,0,13:24])
print(x[0,0,24:35])
print(x[0,0,35:46])
print(x[0,0,46:57])
print(x[0,0,57:61])

[-4.32043412 16.38954033]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 0.          0.          0.          0.          0.          1.
 39.47       -2.93       39.57860281 -5.1        -1.5       ]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 3.728  0.    -2.662 -5.482]


Input Data 標準化

Some parameters

In [0]:
# BATCH_START = 0
TIME_STEPS = x.shape[1]     # 5
BATCH_SIZE = 20
# INPUT_DIM = x.shape[2]      # 36
# EMBED_DIM = 10
# OUTPUT_DIM = y.shape[2]     # 
UNITS = 32      # hidden layer unit數
LR = 1.25E-4
EPOCHS = 50

Constructing the model

In [0]:
input_layers = []
fully_connect_layers = []

# Input: v_lon, v_lat
v_input = Input(shape=(TIME_STEPS, 2), name='v_input')
# v_fc = Dense(5)(v_input)
# v_fc = BatchNormalization()(v_fc)
# v_fc = LeakyReLU(alpha=0.3)(v_fc)
input_layers.append(v_input)
fully_connect_layers.append(v_input)

# Input: type(6), rx, ry, space, rv_lon, rv_lat
for direction in ['f', 'lf', 'rf', 'lr', 'rr']:
    _in = Input(shape=(TIME_STEPS, 11), name='{}_in'.format(direction))
    fc1 = Dense(6, name="{}_fc1".format(direction))(_in)
    fc1 = BatchNormalization()(fc1)
    fc1 = LeakyReLU(alpha=0.3)(fc1)
    fc2 = Dense(3, name="{}_fc2".format(direction))(fc1)
    fc2 = BatchNormalization()(fc2)
    fc2 = LeakyReLU(alpha=0.3)(fc2)
    input_layers.append(_in)
    fully_connect_layers.append(fc2)

# Input: curb, , slow_mix, mix_fast, center (road infrastructure)
infras_in = Input(shape=(TIME_STEPS, 4), name='infras_in')
input_layers.append(infras_in)
fully_connect_layers.append(infras_in)

fully_connect1 = concatenate(fully_connect_layers, name="fully_connect1")
fully_connect1 = BatchNormalization()(fully_connect1)
fully_connect1 = LeakyReLU(alpha=0.3)(fully_connect1)

# processed_sequences = TimeDistributed(model)(input_sequences)

lstm = LSTM(units=UNITS)(fully_connect1)

fc3 = Dense(6, name="{}_fc3".format(direction))(lstm)
fc3 = BatchNormalization()(fc3)
fc3 = LeakyReLU(alpha=0.3)(fc3)
fc4 = Dense(2, name="{}_fc4".format(direction))(fc3)
# fc4 = BatchNormalization()(fc4)
# fc4 = LeakyReLU(alpha=0.3)(fc4)

# predictions = Dense(units=OUTPUT_DIM)(fc_output1)      # model output: v_lon, v_lat

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=input_layers, outputs=fc4)

In [21]:
"""
# Example
# Data
data1 = np.random.rand(3, 5, 2)
data2 = np.random.rand(3, 5, 11)
data3 = np.random.rand(3, 5, 11)
data4 = np.random.rand(3, 5, 11)
data5 = np.random.rand(3, 5, 11)
data6 = np.random.rand(3, 5, 11)
data7 = np.random.rand(3, 5, 4)
"""
x_a = x[:,:,0:2]
x_f = x[:,:,2:13]
x_lf = x[:,:,13:24]
x_rf = x[:,:,24:35]
x_lr = x[:,:,35:46]
x_rr = x[:,:,46:57]
x_infras = x[:,:,57:61]
y = y.reshape(8507, 2)

print(x_a.shape)
print(y.shape)

(8507, 5, 2)
(8507, 2)


In [0]:
result1 = model.predict([x_a, x_f, x_lf, x_rf, x_lr, x_rr, x_infras])
# result2 = model.predict(data2)
print(result1.shape)#; print(result2.shape)

(8507, 2)


In [15]:
from keras.utils import plot_model

# plot_model(model, to_file='./model_lstm.png')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
f_in (InputLayer)               (None, 5, 11)        0                                            
__________________________________________________________________________________________________
lf_in (InputLayer)              (None, 5, 11)        0                                            
__________________________________________________________________________________________________
rf_in (InputLayer)              (None, 5, 11)        0                                            
__________________________________________________________________________________________________
lr_in (InputLayer)              (None, 5, 11)        0                                            
____________________________________________________________________________________________

Loss function: x, y座標的 loss

In [0]:
from keras import backend as K

def loss_function(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))    # RMSE

In [0]:
model.compile(
    loss=loss_function, 
    optimizer=Adam(LR)
)

Training log file

In [0]:
from keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint

csv_logger = CSVLogger(
    filename="./model_lstm_fit.log",
    separator=',',
    append=True
)

In [0]:
# 如果 超過 15次 的循環 在 loss的 收歛上 - 沒有改善 - 停止訓練 

early_stop = EarlyStopping(monitor='val_loss', 
                           min_delta=0.001, 
                           patience=15, # 如果 超過 15次 的循環 在 loss的 收歛上 - 沒有改善 - 停止訓練 
                           mode='min', 
                           verbose=1)

# 每次的訓練循 都去比較模型的 loss 是否有改善, 有就把模型的權重 儲存下來

checkpoint = ModelCheckpoint('model_lstm.h5', 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True, 
                             mode='min', 
                             period=1)

Training

In [22]:
history = model.fit(
    [x_a, x_f, x_lf, x_rf, x_lr, x_rr, x_infras],
    y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[csv_logger],
    validation_split=0.2
)

Train on 6805 samples, validate on 1702 samples
Epoch 1/50
6805/6805 [==============================] - 15s 2ms/step - loss: 7.9326 - val_loss: 5.9337
Epoch 2/50
6805/6805 [==============================] - 15s 2ms/step - loss: 7.6605 - val_loss: 5.7722
Epoch 3/50
6805/6805 [==============================] - 15s 2ms/step - loss: 7.4416 - val_loss: 5.6748
Epoch 4/50
6805/6805 [==============================] - 14s 2ms/step - loss: 7.2201 - val_loss: 5.5608
Epoch 5/50
6805/6805 [==============================] - 14s 2ms/step - loss: 6.9906 - val_loss: 5.3537
Epoch 6/50
6805/6805 [==============================] - 15s 2ms/step - loss: 6.7475 - val_loss: 5.2529
Epoch 7/50
6805/6805 [==============================] - 15s 2ms/step - loss: 6.4880 - val_loss: 5.0296
Epoch 8/50
6805/6805 [==============================] - 15s 2ms/step - loss: 6.2189 - val_loss: 4.9087
Epoch 9/50
6805/6805 [==============================] - 15s 2ms/step - loss: 5.9347 - val_loss: 4.6890
Epoch 10/50
6805/6805 [==